# This file compares the teamtat annotation with Extraction performed (Json)

In [1]:
from sklearn.metrics import precision_score, recall_score, f1_score
from difflib import SequenceMatcher
import numpy as np
import json
import os
import xml.etree.ElementTree as ET 
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity

## File Preparation

In [2]:
def str_toJson(string):
    ##The json output from annotation dataframe was not in correct json format
    # We will change the None to null
    json_string = string.replace("None", "null")

    try:
        # Try to load the JSON string
        json_object = json.loads(json_string)
        return json_object
    except json.JSONDecodeError as e:
        # Catch JSONDecodeError if the string is not valid JSON
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        # Catch any other exceptions
        print(f"An error occurred: {e}")
        return None

In [3]:
def include_passivating(dictionary):
    ##In extraction json, realized that some extraction has passivating molecule that is NOT included in its stability testing. 
    ## Since passivating molecule (if exist) needs to be in stability testing (nexted dictionary), we will transfer the information and spit out a cleaned dictionary. 
    if "passivating_molecule" in dictionary.keys():
        passivating = dictionary['passivating_molecule']
        del dictionary['passivating_molecule']
        
        for entity in dictionary.keys():
            if entity.startswith('test'):
                # print(i['entity'])
                if type(dictionary[entity]) == dict:
                    if 'passivating_molecule' in dictionary[entity].keys():
                        continue
                    else:
                        # print("Have to include passivating molecule in tests")
                        dictionary[entity]['passivating_molecule'] = passivating
        
    return dictionary

In [4]:
## Convert all numerical data into float for both
def convert_numeric(dictionary):
    numerical_key = ['temperature', 'time', 'humidity', 'efficiency_cont', 'efficiency_tret', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']

    translation_table = str.maketrans('', '', 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!@#$%^&*()')
    for key in dictionary.keys():
        if (key.startswith('test')) & (type(dictionary[key]) == dict):
            for entity in dictionary[key].keys():
                if entity in numerical_key:
                    # print(dictionary[key][entity])
                    if isinstance(dictionary[key][entity], str): 
                        substitute = re.sub(r'[^0-9.]', '', dictionary[key][entity][:4])
                        if len(substitute) != 0:
                            numerical_value = float(substitute)
                            dictionary[key][entity] = numerical_value
                        else:
                            dictionary[key][entity] = None
    return dictionary

In [5]:
def convert_efficiency(dictionary):
    entity_decimal = ['efficiency_cont','efficiency_tret']
    for key in dictionary.keys():
        if (key.startswith('test')) & (type(dictionary[key]) == dict):
            for entity in dictionary[key].keys():
                if (entity in entity_decimal) and (dictionary[key][entity] != None):
                    if dictionary[key][entity] == dictionary[key][entity] > 1:
                        dictionary[key][entity] = dictionary[key][entity] / 100
    return dictionary


    

#### Analyzing these outputs

Annotation notes: 
- THE 4 basic variable that is to compare is PEROVSKITE COMPOSITION, ETL, HTL, STRUCTURE
- Stability entity: efficiency_control is wrong, All value is None, so ignore. 
- Common entity: ['stability_type', 'passivating_molecule', 'humidity', 'temperature', 'time', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc', 'efficiency_tret'] 
    - 'efficiency_cont' are included or not

- the efficiency in extracted data need to be converted to decimals since it is in percentage SOMETIMES
    - 'control_efficiency' and 'treatment_efficiency can be ignored


Extraction notes:
- some extraction has passivating molecule that is NOT included in its stability testing. 

### Loading Teamtat Annotation as dataframe

In [6]:
#Teamtat Annotation
annotation_df = pd.read_csv("data/150_papers_json.csv")[["id", "first_num", "output"]]
annotation_df = annotation_df.sort_values(by = ['first_num'])

In [7]:
##Change the format (minor) to be converted to json
annotation_df['output'] = annotation_df['output'].apply(str_toJson)
annotation_df

,id,first_num,output
0,0_54,0,{'perovskite_composition': 'Cs 0.05 FA 0.85 MA...
1,1_22,1,"{'perovskite_composition': None, 'electron_tra..."
2,2_75,2,{'perovskite_composition': 'dibutylammonium le...
3,3_52,3,{'perovskite_composition': 'Cs0.05(MA0.10FA0.8...
4,4_26,4,{'perovskite_composition': '(MAPbBr3)0.05(FAPb...
...,...,...,...
144,145_31,145,{'perovskite_composition': 'Cs0.05(MA0.17FA0.8...
145,146_36,146,{'perovskite_composition': 'Cs0.05(MA0.05FA0.9...
146,147_41,147,{'perovskite_composition': 'formamidinium lead...
147,148_26,148,{'perovskite_composition': 'Cs0.05FA0.85MA0.10...


In [8]:
annotation_df['output'] = annotation_df['output'].apply(convert_numeric)

### Loading in JSON extraction

In [9]:
## extraction performed by Llama (Daniel)
# Read JSON from a file
with open("data/deepseek_updateprompt_6.json", 'r') as f:
    extraction = json.load(f)

extraction_df = pd.DataFrame(list(extraction.items()), columns=['paper_num', 'output'])
extraction_df['paper_num'] = pd.to_numeric(extraction_df['paper_num'])
extraction_df = extraction_df.sort_values('paper_num')
extraction_df


,paper_num,output
8,2,"{'perovskite_composition': '(BA)₂PbI₄', 'elect..."
7,30,"{'perovskite_composition': 'FAI', 'electron_tr..."
5,37,{'perovskite_composition': 'Cs0.2FA0.8Pb(I0.6B...
1,77,{'perovskite_composition': 'FA 0.9 Cs 0.1 PbI ...
3,79,{'perovskite_composition': 'FAPbI3)0.97 (MAPbB...
0,103,"{'perovskite_composition': 'FAPbI3', 'electron..."
2,104,{'perovskite_composition': 'FA-based perovskit...
9,131,"{'perovskite_composition': None, 'electron_tra..."
4,138,{'perovskite_composition': 'Cs 0.05 MA 0.10 FA...
6,143,{'perovskite_composition': '(FAPbI3)0.77 (MAPb...


In [10]:
extraction_df['output'] = extraction_df['output'].apply(include_passivating)
extraction_df

,paper_num,output
8,2,"{'perovskite_composition': '(BA)₂PbI₄', 'elect..."
7,30,"{'perovskite_composition': 'FAI', 'electron_tr..."
5,37,{'perovskite_composition': 'Cs0.2FA0.8Pb(I0.6B...
1,77,{'perovskite_composition': 'FA 0.9 Cs 0.1 PbI ...
3,79,{'perovskite_composition': 'FAPbI3)0.97 (MAPbB...
0,103,"{'perovskite_composition': 'FAPbI3', 'electron..."
2,104,{'perovskite_composition': 'FA-based perovskit...
9,131,"{'perovskite_composition': None, 'electron_tra..."
4,138,{'perovskite_composition': 'Cs 0.05 MA 0.10 FA...
6,143,{'perovskite_composition': '(FAPbI3)0.77 (MAPb...


In [11]:
extraction_df['output'] = extraction_df['output'].apply(convert_numeric)
annotation_df['output'] = annotation_df['output'].apply(convert_numeric)


In [12]:
extraction_df['output'] = extraction_df['output'].apply(convert_efficiency)

## Merging dataframe

In [13]:
evaluate_df = annotation_df.merge(extraction_df, left_on='first_num', right_on='paper_num')[["paper_num", "output_x",'output_y']]
evaluate_df.columns = ['paper_num', 'annotation', 'extracted']
evaluate_df

,paper_num,annotation,extracted
0,2,{'perovskite_composition': 'dibutylammonium le...,"{'perovskite_composition': '(BA)₂PbI₄', 'elect..."
1,30,{'perovskite_composition': 'ammonia healed for...,"{'perovskite_composition': 'FAI', 'electron_tr..."
2,37,"{'perovskite_composition': None, 'electron_tra...",{'perovskite_composition': 'Cs0.2FA0.8Pb(I0.6B...
3,77,"{'perovskite_composition': 'FA0.9Cs0.1Pb3', 'e...",{'perovskite_composition': 'FA 0.9 Cs 0.1 PbI ...
4,79,{'perovskite_composition': '(FAPbI3)0.97(MAPbB...,{'perovskite_composition': 'FAPbI3)0.97 (MAPbB...
5,103,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...,"{'perovskite_composition': 'FAPbI3', 'electron..."
6,104,{'perovskite_composition': 'Methylammonium Lea...,{'perovskite_composition': 'FA-based perovskit...
7,131,"{'perovskite_composition': None, 'electron_tra...","{'perovskite_composition': None, 'electron_tra..."
8,138,{'perovskite_composition': 'Cs0.05MA0.10FA0.85...,{'perovskite_composition': 'Cs 0.05 MA 0.10 FA...
9,143,{'perovskite_composition': '(FAPbI3)0.77(MAPbB...,{'perovskite_composition': '(FAPbI3)0.77 (MAPb...


In [14]:
for row in evaluate_df.itertuples():
    label_value = row.annotation
    extracted_value = row.extracted

    print(label_value)
    print(extracted_value)

{'perovskite_composition': 'dibutylammonium lead iodide', 'electron_transport_layer': 'tin dioxide', 'hole_transport_layer': 'Spiro-OMeTAD', 'structure_pin_nip': 'NIP', 'test_1': {'stability_type': 'ISOSL', 'passivating_molecule': 'butylamine', 'humidity': 85.0, 'temperature': 25.0, 'time': 1620.0, 'control_pce': None, 'treated_pce': 24.3, 'control_voc': None, 'treated_voc': 1.18, 'efficiency_control': None, 'efficiency_tret': 0.98, 'efficiency_cont': 0.58}, 'test_1_2': {'stability_type': 'ISOSD', 'passivating_molecule': None, 'humidity': 85.0, 'temperature': 85.0, 'time': 1056.0, 'control_pce': None, 'treated_pce': 21.3, 'control_voc': None, 'treated_voc': None, 'efficiency_control': None, 'efficiency_tret': 0.94}, 'test_2': {'stability_type': 'ISOSLT', 'passivating_molecule': None, 'humidity': None, 'temperature': 25.0, 'time': 1620.0, 'control_pce': None, 'treated_pce': 24.0, 'control_voc': None, 'treated_voc': None, 'efficiency_control': None, 'efficiency_tret': 0.98}}
{'perovskite

In [15]:
evaluate_df["annotation"][6]

{'perovskite_composition': 'Methylammonium Lead Bromide',
 'electron_transport_layer': 'tin dioxide',
 'hole_transport_layer': None,
 'structure_pin_nip': None,
 'test_1': {'stability_type': 'ISOSLT',
  'passivating_molecule': None,
  'humidity': None,
  'temperature': None,
  'time': 3600.0,
  'control_pce': None,
  'treated_pce': 25.2,
  'control_voc': None,
  'treated_voc': None,
  'efficiency_control': None,
  'efficiency_tret': 1.0},
 'test_2': {'stability_type': 'ISOSLT',
  'passivating_molecule': None,
  'humidity': None,
  'temperature': None,
  'time': 500.0,
  'control_pce': None,
  'treated_pce': None,
  'control_voc': None,
  'treated_voc': None,
  'efficiency_control': None,
  'efficiency_tret': 0.8}}

In [16]:
evaluate_df["extracted"][1]

{'perovskite_composition': 'FAI',
 'electron_transport_layer': 'SnO2',
 'structure_pin_nip': 'PIN',
 'hole_transport_layer': 'Spiro-OMeTAD',
 'test_1': {'stability_type': 'TRPL',
  'temperature': None,
  'time': 103.0,
  'humidity': None,
  'efficiency_cont': 0.065,
  'efficiency_tret': 0.5845,
  'passivating_molecule': 'NH3',
  'control_pce': 6.5,
  'control_voc': 1.16,
  'treated_pce': 58.45,
  'treated_voc': 1.16}}

## Evaluation

- We need precision and recall for EACH variable
- For each variable, calculate the F1 score - There is F1 score for each variable
- Take a weighted average ***For now, just take the average.

In [17]:
def tests_comparison(stability_annotated, label_dict, stability_extracted, extract_dict):
    # print(stability_annotated, label_dict, stability_extracted, extract_dict)
    stability_entity_annotated = ['stability_type', 'passivating_molecule', 'temperature', 'time', 'humidity', 'efficiency_cont', 'efficiency_tret', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    stability_entity_extracted = ['test_name', 'passivating_molecule', 'temperature', 'time', 'humidity', 'control_efficiency', 'treatment_efficiency', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    
    compared_metric = []
    numeric_data_annotated = []
    numeric_data_extracted = []
    for entity_i in range(len(stability_entity_annotated)):
        if entity_i <= 1:
            if stability_entity_extracted[entity_i] not in extract_dict.keys():
                extract_dict[stability_entity_extracted[entity_i]] = None

            if (label_dict[stability_entity_annotated[entity_i]] == None) | (extract_dict[stability_entity_extracted[entity_i]] == None):
                compared_metric.append(None)
            else:
                ##Text entity, perform Sequence Matcher 
                compared = SequenceMatcher(None, label_dict[stability_entity_annotated[entity_i]], extract_dict[stability_entity_extracted[entity_i]]).ratio()
                # print(compared)
                if entity_i == 0:
                    if compared > 0.9:
                        compared_metric.append(1)
                    else:
                        compared_metric.append(0)
                else:
                    compared_metric.append(compared)
        else:
            if stability_entity_extracted[entity_i] not in extract_dict.keys():
                extract_dict[stability_entity_extracted[entity_i]] = 0
            elif extract_dict[stability_entity_extracted[entity_i]] == None:
                extract_dict[stability_entity_extracted[entity_i]] = 0

            if stability_entity_annotated[entity_i] not in label_dict.keys():
                label_dict[stability_entity_annotated[entity_i]] = 0
            elif label_dict[stability_entity_annotated[entity_i]] == None:
                label_dict[stability_entity_annotated[entity_i]] = 0

                
            numeric_data_annotated.append(label_dict[stability_entity_annotated[entity_i]])
            numeric_data_extracted.append(extract_dict[stability_entity_extracted[entity_i]])

    if isinstance(numeric_data_extracted[0], list):
        ##There was one column with two temperature recorded as a list (probably thermal cycling)
        numeric_data_extracted[0] = numeric_data_extracted[0][1]
    cos_sim = cosine_similarity([numeric_data_annotated], [numeric_data_extracted])
    compared_metric.append(cos_sim[0][0])
    
    return compared_metric    

In [31]:
def entity_comparison(entity, label, extracted_dict, text_similarity_threshold = 0.75, numerical_tolerance = 0.027):
    '''
    The tolarance of 2.7% was what was reasonable looking at the absolute difference
    treated_voc 1.18, 1.149, absolute difference 0.026271186440677895

    The text similarity were set to 75% due to the structure example
    FP, NIP, n-i-p, 0.75
    This should be positive
    
    '''
    text_entity = ['stability_type', 'passivating_molecule']
    numerical_entity = ['temperature', 'time', 'humidity','efficiency_cont', 'efficiency_tret', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']

    if entity in text_entity:
        key_to_check = "test_name" if entity == "stability_type" else entity

        # If the key is missing in the extracted annotation, return False Negative
        if (label[entity]!=None) & (extracted_dict[key_to_check]==None):
            # print(f"FN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
            return "FN"
        elif (label[entity]==None) & (extracted_dict[key_to_check]!=None):
            # print(f"TN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
            return "TN"

        label_data = label.get(entity, "")
        extract_data = extracted_dict.get(key_to_check, "")

        # Convert lists to strings if necessary
        if isinstance(label_data, list):
            label_data = " ".join(map(str, label_data))  # Convert list to string
        if isinstance(extract_data, list):
            extract_data = " ".join(map(str, extract_data))  # Convert list to string

        # Ensure values are strings
        if not isinstance(label_data, str) or not isinstance(extract_data, str):
            # print(f"FP, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
            return "FP"  # If data is still not a string, return False Positive

        # Compute similarity score
        similarity = SequenceMatcher(None, label_data.lower(), extract_data.lower()).ratio()

        if similarity > text_similarity_threshold:
            # print(f"TP,{entity} {label_data}, {extract_data}")
            return 'TP'
        else:
            # print(f"FP,{entity} {label_data}, {extract_data}, {similarity}")
            return "FP"
    else:
        # key_to_check = "control_efficiency" if entity == "efficiency_cont" else ("treatment_efficiency" if entity == "efficiency_tret" else entity)

        # print(f"annotated{label[entity]}")
        # print(f"extracted{extracted_dict[entity]}")
        if extracted_dict[entity] == None:
            extracted_dict[entity] = 0

        # If the key is missing in the extracted annotation, return False Negative
        if (label[entity]!=0) & ((extracted_dict[entity]==0) | (entity not in extracted_dict.keys())):
            # print(f"FN, {label_annotation[id]}, {extraction_annotation[entity]}")
            return "FN"
        elif (label[entity]==0) & (extracted_dict[entity]!=0):
            # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
            return "TN"
        elif (label[entity]==0) & (extracted_dict[entity]==0):
            # print(f"TN, {label_annotation[id]}, {extraction_annotation[entity]}")
            return "TN"


        if isinstance(extracted_dict[entity], list):
            ##There was one column with two temperature recorded as a list (probably thermal cycling)
            extracted_dict[entity] = extracted_dict[entity][1]

        # Apply numerical tolerance check
        if (abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )<= numerical_tolerance:

            # print(f"Numerical differences matched: {entity} {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
            return "TP"  # True Positive: Correct numerical extraction
        else:

            # print(f"Numerical differences no match: {entity}, {label[entity]}, {extracted_dict[entity]}, absolute difference {(abs(label[entity] - extracted_dict[entity])) / (abs(label[entity]) )}")
            return "FP"  # False Positive: Incorrect numerical extraction    

In [32]:
def safe_division(numerator, denominator):
    """Returns division result, or 0 if the denominator is zero."""
    return numerator / denominator if denominator != 0 else 0

In [33]:
def text_comparison(id, label_annotation, extraction_annotation, text_similarity_threshold=0.8):
    """Compares text values using string similarity matching.
    - THE 4 basic variable that is to compare is PEROVSKITE COMPOSITION, ETL, HTL, STRUCTURE
    """

    # Handle special case for structure_pin_nip
    # key_to_check = "pin_nip_structure" if id == "structure_pin_nip" else id

    # If the key is missing in the extracted annotation, return False Negative
    if (label_annotation[id]!=None) & (extraction_annotation[id]==None):
        # print(f"FN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
        return "FN"
    elif (label_annotation[id]==None) & (extraction_annotation[id]!=None):
        # print(f"TN, {label_annotation[id]}, {extraction_annotation[key_to_check]}")
        return "TN"

    label_data = label_annotation.get(id, "")
    if id == 'electron_transport_layer' and label_data == "buckminsterfullerene":
        label_data = 'C60'
    extract_data = extraction_annotation.get(id, "")

    # Convert lists to strings if necessary
    if isinstance(label_data, list):
        label_data = " ".join(map(str, label_data))  # Convert list to string
    if isinstance(extract_data, list):
        extract_data = " ".join(map(str, extract_data))  # Convert list to string

    # Ensure values are strings
    if not isinstance(label_data, str) or not isinstance(extract_data, str):
        # print(f"FP, {label_annotation[id]}, {extraction_annotation[id]}")
        return "FP"  # If data is still not a string, return False Positive

    # Compute similarity score
    similarity = SequenceMatcher(None, label_data.lower(), extract_data.lower()).ratio()

    if similarity > text_similarity_threshold:
        # print(f"TP, {label_data}, {extract_data}, {similarity}")
        return 'TP'
    else:
        # print(f"FP, {label_data}, {extract_data}, {similarity}")
        return "FP"


In [36]:
def compare_json(df):
    """
    Compare labeled and extracted JSON data for correctness.

    TP: Correct value extracted by LLM.
    FN: LLM didn't extract this variable.
    FP: LLM extracted a value, but it was incorrect.
    TN: LLM halucinated and returned value that was not extracted
    """
    
    text_variables = ['perovskite_composition', 'electron_transport_layer', 'hole_transport_layer', 'structure_pin_nip']

    
    stability_entity_annotated = ['stability_type', 'temperature', 'time', 'humidity', 'passivating_molecule', 'efficiency_cont', 'efficiency_tret', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    stability_entity_extracted = ['test_name', 'temperature', 'time', 'humidity', 'passivating_molecule','control_efficiency', 'treatment_efficiency', 'control_pce', 'treated_pce', 'control_voc', 'treated_voc']
    
    # Initialize comparison dictionaries
    text_dict = {var: {"TP": 0, "FP": 0, "FN": 0, "TN": 0} for var in text_variables}
    stability_dict = {var: {"TP": 0, "FP": 0, "FN": 0, "TN": 0} for var in stability_entity_annotated}

    for row in df.itertuples():       
        label_value = row.annotation
        extracted_value = row.extracted

        # print(label_value)
        # print(extracted_value)

        for id, label in label_value.items():
            if ('test' in id) and (isinstance(label_value[id], dict)):
                ##Plan for stability test evaluation
                '''
                For each stability condition in annotation, 
                    Pair them with stability condition in extracted
                        With stability of annotation and extraction, use function tests_comparison that returns how similar 2 stabilities are
                    
                Once all the pair is calculated, find the stability name of extraction that was closest to annotation stability. 

                Using this dictionary, we will increment FN, FP, TN, TP for each element of the entity.
                '''
                matched = 0
                stability_match = {}
                for extract_id, extract_label in extracted_value.items():
                    if ('test' in extract_id) and (isinstance(extracted_value[extract_id], dict)):
                        matched += 1
                        match_list = tests_comparison(id, label, extract_id, extract_label)
                        match_list = [0 if item is None else item for item in match_list]
                        # print(extracted_value[extract_id])
                        # print(match_list)
                        stability_match[extract_id] = match_list
        
                if matched == 0:
                    #No stability were extracted, we will add stability_unmatched
                        ##We need to account for if there was NO stability extracted. 
                    for key in stability_dict:
                        if 'FN' in stability_dict[key]:
                            stability_dict[key]['FN'] += 1
                else:
                    stability_match_mean = {stability: np.mean(lis) for stability, lis in stability_match.items()}
                    max_key = max(stability_match_mean, key=stability_match_mean.get)  
                    # print(extracted_value[max_key])
                    ##Now, I need to compare each entity in that found max_key and fill in that FN, dictionary.
                    for entity in label_value[id].keys():
                        if entity == 'efficiency_control':
                            continue
                        entity_result = entity_comparison(entity, label, extracted_value[max_key])
                        stability_dict[entity][entity_result] += 1  
            else:  
                result = text_comparison(id, label_value, extracted_value)
                text_dict[id][result] += 1


    # Merge all results
    combined_dict = {**text_dict, **stability_dict}
    # print("Performance for each variable in dictionary:", combined_dict)

    # Compute precision, recall, and F1-score
    variable_list, precision_list, recall_list, f1_list = [], [], [], []
    for variable, performance in combined_dict.items():
        TP, FP, FN = performance["TP"], performance["FP"], performance["FN"]
        
        precision = safe_division(TP, TP + FP)
        recall = safe_division(TP, TP + FN)
        f1 = safe_division(2 * precision * recall, precision + recall)

        variable_list.append(variable)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    return combined_dict, variable_list, precision_list, recall_list, f1_list

In [37]:
dict_result, variables, precisions, recalls, f1s = compare_json(evaluate_df)

In [38]:
dict_result

{'perovskite_composition': {'TP': 4, 'FP': 5, 'FN': 0, 'TN': 1},
 'electron_transport_layer': {'TP': 0, 'FP': 5, 'FN': 0, 'TN': 5},
 'hole_transport_layer': {'TP': 3, 'FP': 2, 'FN': 0, 'TN': 5},
 'structure_pin_nip': {'TP': 3, 'FP': 5, 'FN': 0, 'TN': 2},
 'stability_type': {'TP': 0, 'FP': 4, 'FN': 12, 'TN': 0},
 'temperature': {'TP': 7, 'FP': 0, 'FN': 2, 'TN': 7},
 'time': {'TP': 8, 'FP': 3, 'FN': 1, 'TN': 4},
 'humidity': {'TP': 3, 'FP': 0, 'FN': 3, 'TN': 10},
 'passivating_molecule': {'TP': 1, 'FP': 6, 'FN': 1, 'TN': 8},
 'efficiency_cont': {'TP': 0, 'FP': 2, 'FN': 1, 'TN': 13},
 'efficiency_tret': {'TP': 6, 'FP': 8, 'FN': 0, 'TN': 2},
 'control_pce': {'TP': 1, 'FP': 4, 'FN': 0, 'TN': 11},
 'treated_pce': {'TP': 5, 'FP': 6, 'FN': 2, 'TN': 3},
 'control_voc': {'TP': 0, 'FP': 2, 'FN': 0, 'TN': 14},
 'treated_voc': {'TP': 3, 'FP': 3, 'FN': 1, 'TN': 9}}

## Calculate Macro f1 score

In [29]:
def macro_f1(f1_list, weight = None):
    if weight == None:
        #If no weight given, do unweighted average of f1 score
        return sum(f1_list) / len(f1_list)
    total_f1 = 0
    for i in range(len(f1_list)):
        total_f1 += (f1_list[i] * weight[i])
    return total_f1 / sum(weight)
    


In [30]:
## The macro f1 score unweighted
macro_f1(f1s)

0.43757446257446253

### This concludes the pipeline of evaluating extraction quality